In [1]:
import heapq
import pandas as pd
import numpy as np
from collections import Counter
from scipy.io import arff
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import StandardScaler
import os

In [6]:
    def _load_and_prepare_data(self, file_path):
        clean_lines = []
        with open(file_path, "r") as f:
            for line in f:
                if not line.strip().lower().startswith("@input") and not line.strip().lower().startswith("@output"):
                    clean_lines.append(line)
        temp_arff = file_path.replace('.dat', '.arff')
        with open(temp_arff, "w") as f:
            f.writelines(clean_lines)
        data, meta = arff.loadarff(temp_arff)
        df = pd.DataFrame(data)
        # Dynamically find class column (usually last attribute in KEEL datasets)
        class_col = meta.names()[-1] if meta is not None and len(meta.names()) > 0 else None
        if class_col is None or class_col not in df.columns:
            print(f"Error: No class column found in {file_path}. Available columns: {list(df.columns)}")
            raise KeyError(f"No class column found in {file_path}. Available columns: {list(df.columns)}")
        # Decode bytes if necessary
        df[class_col] = df[class_col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
        le = LabelEncoder()
        df['Class_encoded'] = le.fit_transform(df[class_col])
        X = df.drop(columns=[class_col, 'Class_encoded'])
        y = df['Class_encoded'].values.ravel()
        os.remove(temp_arff)
        return X, y

# Modified OPTICS

In [3]:
class ModifiedOPTICS:
    def __init__(self, min_samples=5, max_eps=np.inf, xi=0.05, 
                 min_cluster_size=None, predecessor_correction=True):
        self.min_samples = min_samples
        self.max_eps = max_eps
        self.xi = xi
        self.min_cluster_size = min_cluster_size
        self.predecessor_correction = predecessor_correction

        # outputs
        self.reachability_ = None
        self.core_distances_ = None
        self.predecessor_ = None
        self.ordering_ = None
        self.labels_ = None
        self.ri_ = None  # reliable coefficient

    # === Eq. 9: Reliable Coefficient ===
    def _compute_reliable_coeff(self, X, y):
        ri = np.full(len(X), np.inf)
        for i in range(len(X)):
            mask = y != y[i]   # find nearest neighbor from different class
            if np.any(mask):
                dists = np.linalg.norm(X[i] - X[mask], axis=1)
                ri[i] = np.min(dists)
        return ri

    def fit(self, X, y):
        n = len(X)
        self.labels_ = np.full(n, -1)
        self.reachability_ = np.full(n, np.inf)
        self.predecessor_ = np.full(n, -1, dtype=int)
        self.ordering_ = []

        # === Step 1: Compute reliable coefficients ===
        self.ri_ = self._compute_reliable_coeff(X, y)

        # === Step 2: Compute core distances ===
        nn = NearestNeighbors(n_neighbors=self.min_samples)
        nn.fit(X)
        dists, _ = nn.kneighbors(X)
        self.core_distances_ = dists[:, -1].copy()
        self.core_distances_[self.core_distances_ > self.max_eps] = np.inf

        processed = np.zeros(n, dtype=bool)
        seeds = []
        counter = 0

        for point_idx in range(n):
            if processed[point_idx]:
                continue
            self._expand_cluster_order(X, point_idx, processed, seeds, counter)

        # extract xi clusters
        self._extract_xi_clusters()
        return self

    def _expand_cluster_order(self, X, point_idx, processed, seeds, counter):
        processed[point_idx] = True
        self.ordering_.append(point_idx)

        if self.core_distances_[point_idx] == np.inf:
            return

        neighbors = self._get_neighbors(X, point_idx)
        self._update_seeds(X, point_idx, neighbors, processed, seeds, counter)

        while seeds:
            _, _, current_idx = heapq.heappop(seeds)
            if processed[current_idx]:
                continue

            processed[current_idx] = True
            self.ordering_.append(current_idx)
            self.predecessor_[current_idx] = point_idx

            if self.core_distances_[current_idx] != np.inf:
                nbrs = self._get_neighbors(X, current_idx)
                self._update_seeds(X, current_idx, nbrs, processed, seeds, counter)

    def _get_neighbors(self, X, center):
        dists = np.linalg.norm(X - X[center], axis=1)
        return np.where(dists <= self.max_eps)[0]

    # === Eq. 11: Modified Reachability Distance (ORD) ===
    def _update_seeds(self, X, center_idx, neighbors, processed, seeds, counter):
        for idx in neighbors:
            if processed[idx]:
                continue

            dist = np.linalg.norm(X[center_idx] - X[idx])
            # Modified OPTICS reachability
            new_reach = max(self.core_distances_[center_idx], dist) / max(self.ri_[center_idx], 1e-9)

            if self.reachability_[idx] == np.inf or new_reach < self.reachability_[idx]:
                self.reachability_[idx] = new_reach
                heapq.heappush(seeds, (new_reach, counter, idx))
                counter += 1

    def _extract_xi_clusters(self):
        # Simple xi-based cluster extraction (kept same as baseline)
        n = len(self.reachability_)
        ordering = np.array(self.ordering_, dtype=int)
        R = np.r_[self.reachability_[ordering], np.inf]
        xi_comp = 1.0 - self.xi

        labels_ord = np.full(n, -1, dtype=int)
        lab = 0
        start = 0

        for i in range(1, len(R)):
            if R[i] / R[i - 1] < xi_comp:  # steep up 
                if i - start >= self.min_samples:
                    labels_ord[start:i] = lab
                    lab += 1
                start = i

        labels = np.full(n, -1, dtype=int)
        labels[ordering] = labels_ord
        self.labels_ = labels

    def fit_predict(self, X, y):
        return self.fit(X, y).labels_



# Gaussian Augmenter

In [4]:
class GaussianAugmenter:
    def __init__(self, random_state=42):
        self.random_state = np.random.RandomState(random_state)

    def generate_for_clusters(self, X, clusters, y, ri, cluster_ids, n_samples_per_cluster):
        """
        Generate synthetic samples for the given clusters using Multivariate Gaussian noise.
        Prioritize boundary minority samples (smallest ri).
        """
        synth_rows = []
        synth_labels = []

        for cid, n_to_gen in n_samples_per_cluster.items():
            if n_to_gen <= 0:
                continue

            cluster_mask = (clusters == cid)
            X_cluster = X[cluster_mask]
            y_cluster = y[cluster_mask]
            ri_cluster = ri[cluster_mask]

            # Focus only on minority in this cluster
            minority_mask = (y_cluster == 1)
            X_minority = X_cluster[minority_mask]
            ri_minority = ri_cluster[minority_mask]

            if len(X_minority) < 5:  # stricter cutoff
                continue

            # Select subset (boundary-like minority samples: smallest ri)
            k = max(2, int(0.3 * len(X_minority)))
            k = min(k, len(X_minority))
            # Sort by ri ascending (smallest first = boundary)
            sorted_idx = np.argsort(ri_minority)[:k]
            if isinstance(X_minority, pd.DataFrame):
                Xb = X_minority.iloc[sorted_idx]
            else:
                Xb = X_minority[sorted_idx]

            if Xb.shape[0] < 2:  # covariance needs at least 2
                continue

            mu = Xb.mean(axis=0).values if isinstance(Xb, pd.DataFrame) else Xb.mean(axis=0)
            cov = np.cov(Xb.values if isinstance(Xb, pd.DataFrame) else Xb, rowvar=False)
            cov = np.atleast_2d(cov) + np.eye(cov.shape[0]) * 1e-6  # regularize

            for _ in range(n_to_gen):
                synth_sample = self.random_state.multivariate_normal(mu, cov)
                synth_rows.append(synth_sample)
                synth_labels.append(1)

        if not synth_rows:
            return pd.DataFrame()

        synth_df = pd.DataFrame(synth_rows, columns=X.columns)
        synth_df["target"] = synth_labels
        return synth_df

In [ ]:
class MainMethod:
    """
    Main class to run the augmentation and evaluation pipeline,
    reporting only overall, minority, and majority accuracy metrics.
    """
    def __init__(self, random_state=42):
        self.random_state = random_state

    def _load_and_prepare_data(self, file_path):
        clean_lines = []
        with open(file_path, "r") as f:
            for line in f:
                if not line.strip().lower().startswith("@input") and not line.strip().lower().startswith("@output"):
                    clean_lines.append(line)
        temp_arff = file_path.replace('.dat', '.arff')
        with open(temp_arff, "w") as f:
            f.writelines(clean_lines)
        data, meta = arff.loadarff(temp_arff)
        df = pd.DataFrame(data)
        # Dynamically find class column (usually last attribute in KEEL datasets)
        class_col = meta.names()[-1] if meta is not None and len(meta.names()) > 0 else None
        if class_col is None or class_col not in df.columns:
            print(f"Error: No class column found in {file_path}. Available columns: {list(df.columns)}")
            raise KeyError(f"No class column found in {file_path}. Available columns: {list(df.columns)}")
        # Decode bytes if necessary
        df[class_col] = df[class_col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
        le = LabelEncoder()
        df['Class_encoded'] = le.fit_transform(df[class_col])
        X = df.drop(columns=[class_col, 'Class_encoded'])
        y = df['Class_encoded'].values.ravel()
        os.remove(temp_arff)
        return X, y

    def _compute_metrics(self, y_true, y_pred, y_proba=None):
        '''
        Only computes and returns:
            overall_accuracy, minority_accuracy, majority_accuracy
        '''
        acc = accuracy_score(y_true, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        # Assuming binary classification: minority class = 1, majority = 0
        majority_true = (y_true == 0).sum()
        minority_true = (y_true == 1).sum()
        majority_correct = tn
        minority_correct = tp
        majority_acc = majority_correct / majority_true if majority_true > 0 else 0
        minority_acc = minority_correct / minority_true if minority_true > 0 else 0
        return {
            'overall_accuracy': acc,
            'majority_accuracy': majority_acc,
            'minority_accuracy': minority_acc
        }

    def _print_metrics(self, metrics, title):
        print(f"\n{'=' * 30} {title} {'=' * 30}")
        print(f"{'Metric':<20} {'Value':<10}")
        print("-" * 35)
        for metric, value in metrics.items():
            if value is not None:
                print(f"{metric:<20} {value:.3f}")

    def _print_cluster_stats(self, X_train_df, selected_clusters, n_samples_per_cluster):
        """
        Print statistics for all clusters, including noise cluster (-1), regardless of min_minority_in_cluster eligibility.
        """
        print(f"\n{'=' * 30} Cluster Statistics (Noise as Cluster -1) {'=' * 30}")
        # Calculate % Noise
        noise_mask = (X_train_df["cluster"] == -1)
        noise_size = len(X_train_df[noise_mask])
        total_size = len(X_train_df)
        percent_noise = (noise_size / total_size * 100) if total_size > 0 else 0.0
        print(f"% Noise: {percent_noise:.2f}%")
        print(f"{'Cluster ID':<12} {'Minority':<10} {'Majority':<10} {'Ratio':<10} {'Samples to Gen':<15} {'Size':<10}")
        print("-" * 67)
        # Get all unique cluster IDs, including -1
        all_clusters = np.unique(X_train_df["cluster"])
        for cid in sorted(all_clusters, key=lambda x: (x != -1, x)):
            mask = (X_train_df["cluster"] == cid)
            n_minority = sum(X_train_df[mask]["target"] == 1)
            n_majority = sum(X_train_df[mask]["target"] == 0)
            ratio = n_minority / max(n_majority, 1)
            cluster_size = n_minority + n_majority
            n_to_gen = n_samples_per_cluster.get(cid, 0)
            print(f"{cid:<12} {n_minority:<10} {n_majority:<10} {ratio:.3f}    {n_to_gen:<15} {cluster_size:<10}")
        print("-" * 67)
        total_minority = sum(X_train_df["target"] == 1)
        total_majority = sum(X_train_df["target"] == 0)
        total_ratio = total_minority / max(total_majority, 1)
        total_size = total_minority + total_majority
        total_samples_to_gen = sum(n_samples_per_cluster.values())
        print(f"{'Total':<12} {total_minority:<10} {total_majority:<10} {total_ratio:.3f}    {total_samples_to_gen:<15} {total_size:<10}")

    def run_fold(self,fold_no,train_file, test_file,
                 optics_min_samples=7, optics_xi=0.05,
                 ratio_threshold=0.6, min_minority_in_cluster=10, alpha=4.0):
        X_train, y_train = self._load_and_prepare_data(train_file)
        X_test, y_test = self._load_and_prepare_data(test_file)
        print(f"\n{'=' * 30} Processing Fold-{fold_no}: Train size={len(X_train)}, Test size={len(X_test)} {'=' * 30}")

        # Before augmentation
        clf = RandomForestClassifier(random_state=self.random_state)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_proba = clf.predict_proba(X_test)[:, 1]
        before_metrics = self._compute_metrics(y_test, y_pred, y_proba)
        self._print_metrics(before_metrics, "Before Augmentation")

        # Clustering
        scaler = StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        optics = ModifiedOPTICS(min_samples=optics_min_samples, xi=optics_xi, max_eps=np.inf)
        clusters = optics.fit_predict(X_train_scaled, y_train)
        X_train_df = pd.DataFrame(X_train, columns=X_train.columns)
        X_train_df["target"] = y_train
        X_train_df["cluster"] = clusters

        selected_clusters = np.unique(clusters[clusters >= 0])
        noise_mask = (X_train_df["cluster"] == -1)
        n_min_noise = sum(X_train_df[noise_mask]["target"] == 1)
        n_maj_noise = sum(X_train_df[noise_mask]["target"] == 0)
        if n_min_noise >= min_minority_in_cluster:
            selected_clusters = np.append(selected_clusters, -1)

        # Determine samples to generate
        augmenter = GaussianAugmenter(random_state=self.random_state)
        n_samples_per_cluster = {}
        for cid in selected_clusters:
            mask = (X_train_df["cluster"] == cid)
            n_minority = sum(X_train_df[mask]["target"] == 1)
            n_majority = sum(X_train_df[mask]["target"] == 0)
            ratio = n_minority / max(n_majority, 1)
            # print(f"Cluster {cid}: Minority={n_minority}, Majority={n_majority}, Ratio={ratio:.3f}")
            if n_minority < min_minority_in_cluster:
                n_samples_per_cluster[cid] = 0
                continue
            if ratio < ratio_threshold:
                n_samples_per_cluster[cid] = 0
                continue
            n_samples_per_cluster[cid] = int(alpha * n_minority)

        # Print cluster stats (optional, can be commented)
        self._print_cluster_stats(X_train_df, selected_clusters, n_samples_per_cluster)

        # Generate synthetic samples
        synth_df = augmenter.generate_for_clusters(
            X=X_train_df.drop(columns=["target", "cluster"]),
            clusters=X_train_df["cluster"].values,
            y=X_train_df["target"].values,
            ri=optics.ri_,
            cluster_ids=selected_clusters,
            n_samples_per_cluster=n_samples_per_cluster,
        )

        if not synth_df.empty:
            maj = (y_train == 0).sum()
            min_now = (y_train == 1).sum()
            max_add = max(0, int(1.0 * maj - min_now))
            if len(synth_df) > max_add:
                synth_df = synth_df.iloc[:max_add]
            X_train_aug = pd.concat([
                X_train_df.drop(columns=["target", "cluster"]),
                synth_df.drop(columns=["target"])
            ])
            y_train_aug = np.concatenate([
                X_train_df["target"].values,
                synth_df["target"].values
            ])
            print(f"Generated synthetic samples: {len(synth_df)}")
        else:
            X_train_aug = X_train_df.drop(columns=["target", "cluster"])
            y_train_aug = X_train_df["target"].values
            print("No synthetic samples generated.")

        # Class distribution
        before_counts = Counter(y_train)
        after_counts = Counter(y_train_aug)
        minority_class = 1
        majority_class = 0
        before_ratio = before_counts[majority_class]/before_counts[minority_class] if before_counts[majority_class] > 0 else 0
        after_ratio = after_counts[majority_class]/after_counts[minority_class]  if after_counts[majority_class] > 0 else 0
        print(f"\nClass Distribution:")
        print(f"Before - Minority: {before_counts[minority_class]}, Majority: {before_counts[majority_class]}, Ratio: {before_ratio:.3f}")
        print(f"After  - Minority: {after_counts[minority_class]}, Majority: {after_counts[majority_class]}, Ratio: {after_ratio:.3f}")

        # After augmentation
        clf = RandomForestClassifier(random_state=self.random_state)
        clf.fit(X_train_aug, y_train_aug)
        y_pred = clf.predict(X_test)
        y_proba = clf.predict_proba(X_test)[:, 1]
        after_metrics = self._compute_metrics(y_test, y_pred, y_proba)
        self._print_metrics(after_metrics, "After Augmentation")

        return before_metrics, after_metrics

    def _print_average_metrics(self, avg_before, avg_after, avg_gain):
        print(f"\n{'=' * 30} Average Results Across Folds {'=' * 30}")
        print(f"{'Metric':<20} {'Before':<10} {'After':<10} {'Gain':<10} {'Relative % Gain':<15}")
        print("-" * 65)
        for metric in avg_before.keys():
            if avg_before[metric] is not None:
                relative_gain = (avg_gain[metric] / avg_before[metric] * 100) if avg_before[metric] != 0 else 0.0
                print(f"{metric:<20} {avg_before[metric]:.3f}     {avg_after[metric]:.3f}     {avg_gain[metric]:.3f}     {relative_gain:.2f}%")

    def run_5fold_cv(self, fold_prefix='ecoli1-5-fold/ecoli1-5-'):
        folds = [1, 2, 3, 4, 5]
        before_list = []
        after_list = []
        for fold in folds:
            train_file = f"{fold_prefix}{fold}tra.dat"
            test_file = f"{fold_prefix}{fold}tst.dat"
            before, after = self.run_fold(fold,train_file, test_file,
                                         optics_min_samples=8,
                                         optics_xi=0.05,
                                         ratio_threshold=0.6,
                                         min_minority_in_cluster=6,
                                         alpha=1.0)
            before_list.append(before)
            after_list.append(after)
        # Use only the three focused metrics for output
        avg_before = {k: np.mean([d[k] for d in before_list if d[k] is not None]) for k in before_list[0]}
        avg_after = {k: np.mean([d[k] for d in after_list if d[k] is not None]) for k in after_list[0]}
        avg_gain = {k: avg_after[k] - avg_before[k] for k in avg_before}
        self._print_average_metrics(avg_before, avg_after, avg_gain)

if __name__ == "__main__":
    mm = MainMethod(random_state=42)
    mm.run_5fold_cv()


FileNotFoundError: [Errno 2] No such file or directory: 'ecoli1-5-fold/ecoli1-5-1tra.dat'